### Ingestión del archivo "person.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvio JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
name_schema = StructType(fields= [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])                       

In [0]:
person_schema = StructType(fields= [
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema, True)
])             

In [0]:
persons_df = spark.read \
    .schema(person_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/person.json") 

#### Pase 2 - Renombrar las columnas y añadir nuevas columnas
1. "personId" renombrar a "person_id"
2. Agregar las columnas ingestion_date" y "environment"
3. Agregar la columna "name" a partir de la concateniación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
persons_with_columns_df = persons_df \
    .withColumnRenamed("personId", "person_id") \
    .withColumn("name", 
                concat(
                    col("personName.forename"), 
                    lit(" "), 
                    col("personName.surname")
                )
     ) 

persons_with_columns_df = add_ingestion_date(persons_with_columns_df) \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))


#### Paso 3 - Eliminar columnas no "requeridas"

In [0]:
personal_final_df = persons_with_columns_df.drop(col("personName"))

#### Paso 4 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
#overwrite_partition(personal_final_df, "movie_silver","persons", "file_date")

In [0]:
merge_condition = "tgt.person_id = src.person_id AND tgt.file_date = src.file_date"
merge_delta_lake(personal_final_df, "movie_silver","persons", silver_folder_path, merge_condition, "file_date")
#personal_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.persons")
#personal_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/persons")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.persons
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.persons;

In [0]:
dbutils.notebook.exit("Success")